In [1]:
#NOTE about this version - 
#(1) All implementations are vectorized and have been checked against original implementation.
#(2) eps is added to all spectrograms to get rid of divide by zero error.
#(3) Dividing by freq_bin is removed in the SH implementation - not specified in original paper.
#(4) No NaN problems so the pandas notnull function has been removed everywhere
#(5) At the end - saves computed indices for originalspec+4channels into an npz file

#Has operations to get different scatterplots as functions named exp1_... and so on.

#This program loads data from 4 different classes
#good, bad, human, maybe

#And uses 5 different acoustic indices on num_spec recordings from each
#1. ACI 2. ADI 3. ADI_even 4. SH 5. NDSI

#Each recording is exactly 2 s long

#Final result: a scatter plot for each acoustic index + scatterplot matrix - pandas

#Spectrograms used: original+4 differential channels


In [2]:
#imports
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import hilbert
from scipy.signal import stft
import librosa
import os

#for scatter matrix
import pandas as pd
from pandas.plotting import scatter_matrix

#setting eps value for converting spec to log
eps = 1e-10

#function definitions for acoustic indices begin here


In [3]:
#1. ACI
def compute_aci(specs, time_steps, freq_bins, num_specs):
      
    #Function computes Acoustic Complexity Index
    #According to:
    #http://www.iinsteco.org/people/pubblications/almo/2011/2011_A_new%20methodology_to_infer_the_singing_activity.pdf
    
    #Adding eps
    specs = specs + eps  
    
    abs_diff = np.abs(specs[:, :, 1:] - specs[:, :, 0:time_steps-1])
    ACI = np.sum(np.sum(abs_diff, axis = 2)/np.sum(specs, axis = 2), axis = 1)
    
    return ACI

In [4]:
#2. Acoustic diversity
def compute_adi(specs, time_steps):

    #Function computes Acoustic Diversity Index
    #According to: Pekin (2012)
    #Each bin corresponds to one of the 70 frequency points
   
    #Adding eps
    specs = specs + eps
    
    #Shyam
    row_sums = specs.sum(axis=2, keepdims=False)
    all_sum = row_sums.sum(axis=1, keepdims=True)
    row_norm = row_sums/all_sum                                  #normalization
    ADI_all = (row_norm * np.log(row_norm)).sum(axis=1)

    return ADI_all


In [5]:
#Helper function for compute_adi_even
def get_start_stop_indices(freq_bins, multiples=1000, num_bins=8):
    
    #find the frequency index numbers to group in bins of 1kHz width from 0-1 kHz, 1-2, till 7-8kHz
    stop_freq_idx = np.zeros((num_bins, 1))
    start_freq_idx = np.zeros((num_bins,1))

    for i in range(num_bins):
        stop_freq = multiples*(i+1)
        #to find the index to clip
        stop_freq_idx[i] = freq_bins.searchsorted(stop_freq, side='right') - 1
        if (i<num_bins-1):
            start_freq_idx[i+1] = stop_freq_idx[i]+1
        
    return start_freq_idx, stop_freq_idx
        

In [6]:
#3. Acoustic diversity with evenly spaced bins
def compute_adi_even(num_bins, specs, freq_vals, start_indices, stop_indices):

    #Function computes Acoustic Diversity Index
    #According to: Pekin (2012)
    
    #Adding eps
    specs = specs+eps
  
    num_specs = specs.shape[0]
    ADI_bins = np.zeros((num_specs, num_bins))
    
    ADI_all = 0
    
    for i in range(num_bins):      
    
        #for loop for all time steps in same freq bin - sum each row
        start = int(start_indices[i])
        stop = int(stop_indices[i])
        specs_strip = specs[:, start:stop, :]
        ADI_bins[:, i] = np.sum(np.sum(specs_strip, axis = 2), axis = 1)
        
    #Normalizing by summing along axis 1: bin number
    ADI_bins_norm = ADI_bins/np.sum(ADI_bins, axis = 1, keepdims=True)
    
    #Computing
    ADI_all = (ADI_bins_norm * np.log(ADI_bins_norm)).sum(axis=1)
    
    return ADI_all

In [7]:
#4. Spectral entropy 
def compute_sh(specs, time_steps, freq_bin):
    
    #This function is a modified version of  spectral entropy as described in Sueur (2008)
       
    #Adding eps
    specs = specs+eps
    
    #Convert each frame to absolute value
    time_sums = np.sum(np.abs(specs), axis = 1, keepdims=True)
    spec_norm = np.divide(np.abs(specs), time_sums)

    #Added the eps in the log term, using np.multiply
    mult_term = np.multiply(spec_norm, np.log2(spec_norm + eps))
    
    #print("First frame multiplied: ", mult_term[0, :, 1])
   
    sh_all = - np.sum(mult_term.sum(axis = 1), axis = 1)
    sh_all = sh_all/np.log2(freq_bin)          #normalizing
    
    return sh_all

In [8]:
#5. NDSI
def compute_ndsi(spec, start_indices, stop_indices, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq):
    
    #Taken from the Kasten 2012 paper - page 6 has NDSI description
    
    #Adding eps
    spec = spec + eps
    
    #Anthrophony bin: 1-2 kHz
    start_a = int(start_indices[start_a_freq])
    stop_a = int(stop_indices[stop_a_freq-1])
    
    #Biophony bin: 2-8 kHz
    start_b = int(start_indices[start_b_freq])
    stop_b = int(stop_indices[stop_b_freq-1])
    
    #Taking absolute value - power spectral density was squared, so intensities are made positive here

    anth_sum = np.sum(np.sum(np.abs(spec[:, start_a:stop_a, :]), axis = 2), axis = 1)
    bio_sum = np.sum(np.sum(np.abs(spec[:, start_b:stop_b, :]), axis = 2), axis = 1)
    NDSI = (bio_sum-anth_sum)/(bio_sum+anth_sum)
    
    return NDSI

In [9]:
#function to load spectrograms and display dimensions (optional)
def load_display_dimensions(class_name, str_file, dim=0):

    #   description of data files
    #   spec_data is like a dict with fields
    #   'specs' (contains the mel-filtered spectrograms, in linear scale, as [num_specs, num_rows, num_cols])
    #   'spec_f' (contains the frequency axis points as a 1D array)
    #   'spec_t' (contains the time axis points as a 1D array)   
    
    fname_load = class_name + str_file
    spec_data = np.load(fname_load)
    
    time_steps = len(spec_data['spec_t'])
    freq_bins = len(spec_data['spec_f'])
    
    if (dim==1):
        print('File has %i specs of dimensions (%i x %i)' % (
        spec_data['specs'].shape[0], spec_data['specs'].shape[1], spec_data['specs'].shape[2]))
        print('Frequency axis points are in spec_data[\'spec_f\'] and has %i values' % freq_bins)
        print('Time axis points are in spec_data[\'spec_t\'] and has %i values' % time_steps)
        
    return spec_data, time_steps, freq_bins
    


In [10]:
#function to calculate all acoustic indices and plot spectrogram (optional)
def calc_plot(num_bins, spec_start_idx, num_specs, spec_data, ch, time_steps, freq_bins, freq_vals, start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):
    
    ACI = np.zeros((num_specs, 1))
    ADI = np.zeros((num_specs, 1))
    ADI_even = np.zeros((num_specs, 1))
    SH = np.zeros((num_specs, 1))
    NDSI = np.zeros((num_specs, 1))
       
    if (plot==1):
        fig, axes = plt.subplots(num_specs, 1, sharex='all')
        plt.title('Spectrogram for ' + 'good')
      
    #Store log version of all spectrograms in all_specs
    #all_specs = 10 * np.log10(spec_data['specs'][spec_start_idx:spec_start_idx+num_specs, :, :] + eps)
    
    if(ch==-1):
        #Original spectrogram
        all_specs = 10 * np.log10(spec_data['specs'][spec_start_idx:spec_start_idx+num_specs, :, :] + eps)
    elif (ch==5):
        #Experiment no. 4 - has 3 dimensions only
        all_specs = spec_data
    else:
        #4 differential channels so 1 dimension extra at the end for which channel
        all_specs = spec_data['specs'][spec_start_idx:spec_start_idx+num_specs, :, :, ch] 

    #Vectorized implementations 
    ACI = compute_aci(all_specs, time_steps, freq_bins, num_specs)
    ADI = compute_adi(all_specs, time_steps)
    ADI_even = compute_adi_even(num_bins, all_specs, freq_vals, start_freq_idx, stop_freq_idx)
    SH = compute_sh(all_specs, time_steps, freq_bins)
    NDSI = compute_ndsi(all_specs, start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq)
             
    if(plot==1):
        cax = axes[spec_idx].imshow(current_spec, cmap=plt.get_cmap('jet'), interpolation='none', origin='lower')
        fig.colorbar(cax, ax=axes[spec_idx], orientation='vertical')
        
    
    return ACI, ADI, ADI_even, SH, NDSI


In [11]:
#calls all functions for each class - is for the original data
def calls_orig(class_name, spec_start_idx, num_specs, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):

    #Load the correct file
    str_file = '.npz'
    spec_data, time_steps, freq_bins = load_display_dimensions(class_name, str_file, dim)
    
    #Calling helper function for compute_adi_even
    start_freq_idx, stop_freq_idx = get_start_stop_indices(spec_data['spec_f'], multiples, num_bins)

    #Computing acoustic indices for all 4 channels
    ch = -1
    ACI, ADI, ADI_even, SH, NDSI = calc_plot(num_bins, spec_start_idx, num_specs, spec_data, ch, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    
    return ACI, ADI, ADI_even, SH, NDSI

In [12]:
#calls all functions for each class
def calls(class_name, spec_start_idx, num_specs, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):

    #Load the correct file
    str_file = '_preprocessed_uncompressed.npz'
    spec_data, time_steps, freq_bins = load_display_dimensions(class_name, str_file, dim)
    
    #Calling helper function for compute_adi_even
    start_freq_idx, stop_freq_idx = get_start_stop_indices(spec_data['spec_f'], multiples, num_bins)

    #Computing acoustic indices for all 4 channels
    ACI1, ADI1, ADI_even1, SH1, NDSI1 = calc_plot(num_bins, spec_start_idx, num_specs, spec_data, 0, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    ACI2, ADI2, ADI_even2, SH2, NDSI2 = calc_plot(num_bins, spec_start_idx, num_specs, spec_data, 1, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    ACI3, ADI3, ADI_even3, SH3, NDSI3 = calc_plot(num_bins, spec_start_idx, num_specs, spec_data, 2, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    ACI4, ADI4, ADI_even4, SH4, NDSI4 = calc_plot(num_bins, spec_start_idx, num_specs, spec_data, 3, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)

    return ACI1, ADI1, ADI_even1, SH1, NDSI1, ACI2, ADI2, ADI_even2, SH2, NDSI2, ACI3, ADI3, ADI_even3, SH3, NDSI3, ACI4, ADI4, ADI_even4, SH4, NDSI4
       

In [13]:
#Main - setting all parameters

#Defining classes
class1 = 'good'
class2 = 'bad'
class3 = 'human'
class4 = 'maybe'

#Make sure the indices don't get larger than spec_data['specs'].shape[0]
spec_start_idx = 0
#num_specs = 2

num_specs_g = 2958
num_specs_b = 127
num_specs_h = 205
num_specs_m = 131
'''
num_specs_g = 120
num_specs_b = 120
num_specs_h = 120
num_specs_m = 120'''

#Other parameters
plot = 0
dim = 1
num_bins = 8            #for compute_adi_even
multiples = 1000

#NDSI bin frequencies in kHz - a = anthrophony, b = biophony
#Values set according to the original paper
start_a_freq = 1
stop_a_freq = 2
start_b_freq = 2
stop_b_freq = 8



In [14]:
#Calls for each class
ACI_G1, ADI_G1, ADI_even_G1, SH_G1, NDSI_G1, ACI_G2, ADI_G2, ADI_even_G2, SH_G2, NDSI_G2, ACI_G3, ADI_G3, ADI_even_G3, SH_G3, NDSI_G3, ACI_G4, ADI_G4, ADI_even_G4, SH_G4, NDSI_G4 = calls(class1, spec_start_idx, num_specs_g, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Good")
ACI_B1, ADI_B1, ADI_even_B1, SH_B1, NDSI_B1, ACI_B2, ADI_B2, ADI_even_B2, SH_B2, NDSI_B2, ACI_B3, ADI_B3, ADI_even_B3, SH_B3, NDSI_B3, ACI_B4, ADI_B4, ADI_even_B4, SH_B4, NDSI_B4 = calls(class2, spec_start_idx, num_specs_b, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Bad")
ACI_H1, ADI_H1, ADI_even_H1, SH_H1, NDSI_H1, ACI_H2, ADI_H2, ADI_even_H2, SH_H2, NDSI_H2, ACI_H3, ADI_H3, ADI_even_H3, SH_H3, NDSI_H3, ACI_H4, ADI_H4, ADI_even_H4, SH_H4, NDSI_H4 = calls(class3, spec_start_idx, num_specs_h, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Human")
ACI_M1, ADI_M1, ADI_even_M1, SH_M1, NDSI_M1, ACI_M2, ADI_M2, ADI_even_M2, SH_M2, NDSI_M2, ACI_M3, ADI_M3, ADI_even_M3, SH_M3, NDSI_M3, ACI_M4, ADI_M4, ADI_even_M4, SH_M4, NDSI_M4 = calls(class4, spec_start_idx, num_specs_m, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Maybe")


KeyboardInterrupt: 

In [ ]:
#Calls for each class - original spectrograms
ACI_G, ADI_G, ADI_even_G, SH_G, NDSI_G = calls_orig(class1, spec_start_idx, num_specs_g, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Good")
ACI_B, ADI_B, ADI_even_B, SH_B, NDSI_B = calls_orig(class2, spec_start_idx, num_specs_b, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Bad")
ACI_H, ADI_H, ADI_even_H, SH_H, NDSI_H = calls_orig(class3, spec_start_idx, num_specs_h, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Human")
ACI_M, ADI_M, ADI_even_M, SH_M, NDSI_M = calls_orig(class4, spec_start_idx, num_specs_m, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Maybe")


In [ ]:
#Checking values in vectorized+adding eps implementation - original spec
print(ACI_G)
print(ADI_G)
print(ADI_even_G)
print(SH_G)
print(NDSI_G)

In [ ]:
#Checking values in vectorized+adding eps implementation - channel 1
print(ACI_G1)
print(ADI_G1)
print(ADI_even_G1)
print(SH_G1)
print(NDSI_G1)

In [ ]:
#Seaborn for visualizations
import seaborn as sns
sns.set(style="ticks")

In [ ]:
#General procedure to get pandas dataframes: 
#(1)reshape the array
#(2) create a dataframe with columns as acoustic indices
#(3) add an extra column with class name

def convert_df(ind1, ind2, ind3, ind4, ind5, num_specs, class_type, ch):
    ind1 = np.reshape(ind1, (num_specs, ))
    ind2 = np.reshape(ind2, (num_specs, ))
    ind3 = np.reshape(ind3, (num_specs, ))
    ind4 = np.reshape(ind4, (num_specs, ))
    ind5 = np.reshape(ind5, (num_specs, ))
    
    Class = class_type

    data = pd.DataFrame({'ACI': ind1, 'ADI': ind2, 'ADI_even': ind3, 'SH': ind4,'NDSI': ind5, 'Class': Class}, columns=['ACI', 'ADI', 'ADI_even', 'SH', 'NDSI', 'Class'])
    #print(data.shape)
    #data.head()
    
    #renaming the columns of the dataframe 'data' to be used while combining not NaN rows later
    if(ch!=-1 and ch!=5):
        data.rename(columns={'ACI': 'ACI'+str(ch), 'ADI': 'ADI'+str(ch), 'ADI_even': 'ADI_even'+str(ch), 'SH': 'SH'+str(ch), 'NDSI': 'NDSI'+str(ch), 'Class': 'Class'+str(ch)}, inplace=True)
 
    return data

In [ ]:
def get_df_and_plot(ch, ACI_G, ADI_G, ADI_even_G, SH_G, NDSI_G, num_specs_g, ACI_B, ADI_B, ADI_even_B, SH_B, NDSI_B, num_specs_b, ACI_H, ADI_H, ADI_even_H, SH_H, NDSI_H, num_specs_h, ACI_M, ADI_M, ADI_even_M, SH_M, NDSI_M, num_specs_m):


    #Converting to dataframes 
    data_G_orig = convert_df(ACI_G, ADI_G, ADI_even_G, SH_G, NDSI_G, num_specs_g, 'Good', ch)
    data_B_orig = convert_df(ACI_B, ADI_B, ADI_even_B, SH_B, NDSI_B, num_specs_b, 'Bad', ch)
    data_H_orig = convert_df(ACI_H, ADI_H, ADI_even_H, SH_H, NDSI_H, num_specs_h, 'Human', ch)
    data_M_orig = convert_df(ACI_M, ADI_M, ADI_even_M, SH_M, NDSI_M, num_specs_m, 'Maybe', ch)
    
    #Scatterplot matrix for all 4 classes
    all_data_arr = [data_G_orig, data_B_orig, data_H_orig, data_M_orig]
    all_data_df = pd.concat(all_data_arr)

    print(all_data_df.shape)
    #all_data_df.head()
    
    if(ch!=-1 and ch!=5):
        Hue = 'Class' + str(ch)
    else:
        Hue = 'Class'
    
    sns.pairplot(all_data_df, hue=Hue)
    plt.show()
    
    #Scatterplot matrix for good and bad only
    gb_data_arr = [data_G_orig, data_B_orig]
    gb_data_df = pd.concat(gb_data_arr)

    print(gb_data_df.shape)
    #gb_data_df.head()

    sns.pairplot(gb_data_df, hue=Hue)
    plt.show()
    
    return all_data_df, gb_data_df
    
    

In [ ]:
all_data_df_orig, gb_data_df_orig = get_df_and_plot(-1, ACI_G, ADI_G, ADI_even_G, SH_G, NDSI_G, num_specs_g, ACI_B, ADI_B, ADI_even_B, SH_B, NDSI_B, num_specs_b, ACI_H, ADI_H, ADI_even_H, SH_H, NDSI_H, num_specs_h, ACI_M, ADI_M, ADI_even_M, SH_M, NDSI_M, num_specs_m)

all_data_df_orig.head()

In [ ]:
all_data_df1, gb_data_df1 = get_df_and_plot(1, ACI_G1, ADI_G1, ADI_even_G1, SH_G1, NDSI_G1, num_specs_g, ACI_B1, ADI_B1, ADI_even_B1, SH_B1, NDSI_B1, num_specs_b, ACI_H1, ADI_H1, ADI_even_H1, SH_H1, NDSI_H1, num_specs_h, ACI_M1, ADI_M1, ADI_even_M1, SH_M1, NDSI_M1, num_specs_m)


In [ ]:
all_data_df2, gb_data_df2 = get_df_and_plot(2, ACI_G2, ADI_G2, ADI_even_G2, SH_G2, NDSI_G2, num_specs_g, ACI_B2, ADI_B2, ADI_even_B2, SH_B2, NDSI_B2, num_specs_b, ACI_H2, ADI_H2, ADI_even_H2, SH_H2, NDSI_H2, num_specs_h, ACI_M2, ADI_M2, ADI_even_M2, SH_M2, NDSI_M2, num_specs_m)


In [ ]:
all_data_df3, gb_data_df3 = get_df_and_plot(3, ACI_G3, ADI_G3, ADI_even_G3, SH_G3, NDSI_G3, num_specs_g, ACI_B3, ADI_B3, ADI_even_B3, SH_B3, NDSI_B3, num_specs_b, ACI_H3, ADI_H3, ADI_even_H3, SH_H3, NDSI_H3, num_specs_h, ACI_M3, ADI_M3, ADI_even_M3, SH_M3, NDSI_M3, num_specs_m)

In [ ]:
all_data_df4, gb_data_df4 = get_df_and_plot(4, ACI_G4, ADI_G4, ADI_even_G4, SH_G4, NDSI_G4, num_specs_g, ACI_B4, ADI_B4, ADI_even_B4, SH_B4, NDSI_B4, num_specs_b, ACI_H4, ADI_H4, ADI_even_H4, SH_H4, NDSI_H4, num_specs_h, ACI_M4, ADI_M4, ADI_even_M4, SH_M4, NDSI_M4, num_specs_m)


In [ ]:
#Combining all 4 channels 
all_channel_arr = [all_data_df1, all_data_df2, all_data_df3, all_data_df4]
all_channel_df = pd.concat(all_channel_arr, axis=1)

print(all_channel_df.shape)
all_channel_df.head()


In [ ]:
#doing the same for just good-bad data
good_bad_arr = [gb_data_df1, gb_data_df2, gb_data_df3, gb_data_df4]
good_bad_df = pd.concat(good_bad_arr, axis=1)

print(good_bad_df.shape)
good_bad_df.head()


In [ ]:
#Max function
def get_max_ind(df):
    
    max_df = pd.DataFrame(columns=['ACI','ADI','ADI_even','SH','NDSI','Class'])
    max_df.head()
       
    max_df['ACI'] =       df[['ACI1','ACI2','ACI3','ACI4']].max(axis=1)    
    max_df['ADI'] =       df[['ADI1','ADI2','ADI3','ADI4']].max(axis=1)
    max_df['ADI_even'] =  df[['ADI_even1','ADI_even2','ADI_even3','ADI_even4']].max(axis=1)
    max_df['SH'] =        df[['SH1','SH2','SH3','SH4']].max(axis=1)
    max_df['NDSI'] =      df[['NDSI1','NDSI2','NDSI3','NDSI4']].max(axis=1)
    max_df['Class'] =     df['Class1']
    
    return max_df


In [ ]:
#Average function
def get_avg_ind(df):
    
    avg_df = pd.DataFrame(columns=['ACI','ADI','ADI_even','SH','NDSI','Class'])
    avg_df.head()
    
    avg_df['ACI'] =      (df['ACI1']+df['ACI2']+df['ACI3']+df['ACI4'])/4
    avg_df['ADI'] =      (df['ADI1']+df['ADI2']+df['ADI3']+df['ADI4'])/4
    avg_df['ADI_even'] = (df['ADI_even1']+df['ADI_even2']+df['ADI_even3']+df['ADI_even4'])/4
    avg_df['SH'] =       (df['SH1']+df['SH2']+df['SH3']+df['SH4'])/4
    avg_df['NDSI'] =     (df['NDSI1']+df['NDSI2']+df['NDSI3']+df['NDSI4'])/4
    avg_df['Class'] =     df['Class1']
    
    return avg_df

In [ ]:
#Calling functions to combine the data - (1) Average (2) Max 

#Avg
avg_all_df = get_avg_ind(all_channel_df)
avg_gb_df = get_avg_ind(good_bad_df)

print(avg_all_df.shape)
sns.pairplot(avg_all_df, hue="Class")
plt.show()

print(avg_gb_df.shape)
sns.pairplot(avg_gb_df, hue="Class")
plt.show()

  

In [ ]:
#Max
max_all_df = get_max_ind(all_channel_df)
max_gb_df = get_max_ind(good_bad_df)

print(max_all_df.shape)
sns.pairplot(max_all_df, hue="Class")
plt.show()

print(max_gb_df.shape)
sns.pairplot(max_gb_df, hue="Class")
plt.show()

In [ ]:
#Note - dataframes here are - 
#Combined: (1) original - all_data_df_orig, gb_data_df_orig (2) channels - all_channel_df, good_bad_df
#Separate: channels - all_data_df1, gb_data_df2, etc.

In [ ]:
def combine_specs_4_5(exp, spec_data, spec_start_idx, num_specs):
    
    if(exp==4):
        spec_1 = spec_data['specs'].max(axis=3, keepdims=False)
        
    elif(exp==5):
        spec_1 = spec_data['specs'].mean(axis=3, keepdims=False)

    #print(spec_1.shape)

    return spec_1
    

In [ ]:
#calls all functions for each class - is for Exp4
def calls_exp_4_5(exp, class_name, spec_start_idx, num_specs, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot=0):

    #Load the correct file
    str_file = '_preprocessed_uncompressed.npz'
    spec_data, time_steps, freq_bins = load_display_dimensions(class_name, str_file, dim)
    
    #Calling helper function for compute_adi_even
    start_freq_idx, stop_freq_idx = get_start_stop_indices(spec_data['spec_f'], multiples, num_bins)

    #Combining spectrograms
    new_spec = combine_specs_4_5(exp, spec_data, spec_start_idx, num_specs)
    
    #Computing acoustic indices for all 4 channels
    ch = 5
    print(new_spec.shape)
    
    ACI, ADI, ADI_even, SH, NDSI = calc_plot(num_bins, spec_start_idx, num_specs, new_spec, ch, time_steps, freq_bins, spec_data['spec_f'], start_freq_idx, stop_freq_idx, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
    
    return ACI, ADI, ADI_even, SH, NDSI

In [ ]:
#Calls for each class - Exp 4 with max values

exp = 4
ACI_G_exp4, ADI_G_exp4, ADI_even_G_exp4, SH_G_exp4, NDSI_G_exp4 = calls_exp_4_5(exp, class1, spec_start_idx, num_specs_g, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Good")

ACI_B_exp4, ADI_B_exp4, ADI_even_B_exp4, SH_B_exp4, NDSI_B_exp4 = calls_exp_4_5(exp, class2, spec_start_idx, num_specs_b, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Bad")

ACI_H_exp4, ADI_H_exp4, ADI_even_H_exp4, SH_H_exp4, NDSI_H_exp4 = calls_exp_4_5(exp, class3, spec_start_idx, num_specs_h, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Human")

ACI_M_exp4, ADI_M_exp4, ADI_even_M_exp4, SH_M_exp4, NDSI_M_exp4 = calls_exp_4_5(exp, class4, spec_start_idx, num_specs_m, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Maybe")



In [ ]:
#Plotting for Exp4
all_data_df_exp4, gb_data_df_exp4 = get_df_and_plot(5, ACI_G_exp4, ADI_G_exp4, ADI_even_G_exp4, SH_G_exp4, NDSI_G_exp4, num_specs_g, ACI_B_exp4, ADI_B_exp4, ADI_even_B_exp4, SH_B_exp4, NDSI_B_exp4, num_specs_b, ACI_H_exp4, ADI_H_exp4, ADI_even_H_exp4, SH_H_exp4, NDSI_H_exp4, num_specs_h, ACI_M_exp4, ADI_M_exp4, ADI_even_M_exp4, SH_M_exp4, NDSI_M_exp4, num_specs_m)

all_data_df_exp4.head()

In [ ]:
#Calls for each class - Exp 5 with  avg values

exp = 5
ACI_G_exp5, ADI_G_exp5, ADI_even_G_exp5, SH_G_exp5, NDSI_G_exp5 = calls_exp_4_5(exp, class1, spec_start_idx, num_specs_g, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Good")

ACI_B_exp5, ADI_B_exp5, ADI_even_B_exp5, SH_B_exp5, NDSI_B_exp5 = calls_exp_4_5(exp, class2, spec_start_idx, num_specs_b, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Bad")

ACI_H_exp5, ADI_H_exp5, ADI_even_H_exp5, SH_H_exp5, NDSI_H_exp5 = calls_exp_4_5(exp, class3, spec_start_idx, num_specs_h, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Human")

ACI_M_exp5, ADI_M_exp5, ADI_even_M_exp5, SH_M_exp5, NDSI_M_exp5 = calls_exp_4_5(exp, class4, spec_start_idx, num_specs_m, multiples, num_bins, dim, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq, plot)
print("Done with class Maybe")

In [ ]:
#Plotting for Exp5
all_data_df_exp5, gb_data_df_exp5 = get_df_and_plot(5, ACI_G_exp5, ADI_G_exp5, ADI_even_G_exp5, SH_G_exp5, NDSI_G_exp5, num_specs_g, ACI_B_exp5, ADI_B_exp5, ADI_even_B_exp5, SH_B_exp5, NDSI_B_exp5, num_specs_b, ACI_H_exp5, ADI_H_exp5, ADI_even_H_exp5, SH_H_exp5, NDSI_H_exp5, num_specs_h, ACI_M_exp5, ADI_M_exp5, ADI_even_M_exp5, SH_M_exp5, NDSI_M_exp5, num_specs_m)

all_data_df_exp5.head()

In [ ]:
#Saving ACI values for each class in a separate file

#To access ACI value of channel 2, 5th spec, use:
#ACI_loaded = np.load('ACI_good.npy', mmap_mode='r')
#print(ACI_loaded[2][4])

#Channel 0 refers to original spectrogram

#So given channel_idx and spec_idx (0 onwards), we want ACI_loaded[channel_idx][spec_idx]

ACI_G_vals = (ACI_G, ACI_G1, ACI_G2, ACI_G3, ACI_G4)
np.save('ACI_good.npy', ACI_G_vals)

ACI_B_vals = (ACI_B, ACI_B1, ACI_B2, ACI_B3, ACI_B4)
np.save('ACI_bad.npy', ACI_B_vals)

ACI_H_vals = (ACI_H, ACI_H1, ACI_H2, ACI_H3, ACI_H4)
np.save('ACI_human.npy', ACI_H_vals)

ACI_M_vals = (ACI_M, ACI_M1, ACI_M2, ACI_M3, ACI_M4)
np.save('ACI_maybe.npy', ACI_M_vals)


In [ ]:
#Saving ADI values for each class in a separate file

ADI_G_vals = (ADI_G, ADI_G1, ADI_G2, ADI_G3, ADI_G4)
np.save('ADI_good.npy', ADI_G_vals)

ADI_B_vals = (ADI_B, ADI_B1, ADI_B2, ADI_B3, ADI_B4)
np.save('ADI_bad.npy', ADI_B_vals)

ADI_H_vals = (ADI_H, ADI_H1, ADI_H2, ADI_H3, ADI_H4)
np.save('ADI_human.npy', ADI_H_vals)

ADI_M_vals = (ADI_M, ADI_M1, ADI_M2, ADI_M3, ADI_M4)
np.save('ADI_maybe.npy', ADI_M_vals)

In [ ]:
#Saving ADI_even values for each class in a separate file

ADI_even_G_vals = (ADI_even_G, ADI_even_G1, ADI_even_G2, ADI_even_G3, ADI_even_G4)
np.save('ADI_even_good.npy', ADI_even_G_vals)

ADI_even_B_vals = (ADI_even_B, ADI_even_B1, ADI_even_B2, ADI_even_B3, ADI_even_B4)
np.save('ADI_even_bad.npy', ADI_even_B_vals)

ADI_even_H_vals = (ADI_even_H, ADI_even_H1, ADI_even_H2, ADI_even_H3, ADI_even_H4)
np.save('ADI_even_human.npy', ADI_even_H_vals)

ADI_even_M_vals = (ADI_even_M, ADI_even_M1, ADI_even_M2, ADI_even_M3, ADI_even_M4)
np.save('ADI_even_maybe.npy', ADI_even_M_vals)

In [ ]:
#Saving SH values for each class in a separate file

SH_G_vals = (SH_G, SH_G1, SH_G2, SH_G3, SH_G4)
np.save('SH_good.npy', SH_G_vals)

SH_B_vals = (SH_B, SH_B1, SH_B2, SH_B3, SH_B4)
np.save('SH_bad.npy', SH_B_vals)

SH_H_vals = (SH_H, SH_H1, SH_H2, SH_H3, SH_H4)
np.save('SH_human.npy', SH_H_vals)

SH_M_vals = (SH_M, SH_M1, SH_M2, SH_M3, SH_M4)
np.save('SH_maybe.npy', SH_M_vals)

In [ ]:
#Saving NDSI values for each class in a separate file

NDSI_G_vals = (NDSI_G, NDSI_G1, NDSI_G2, NDSI_G3, NDSI_G4)
np.save('NDSI_good.npy', NDSI_G_vals)

NDSI_B_vals = (NDSI_B, NDSI_B1, NDSI_B2, NDSI_B3, NDSI_B4)
np.save('NDSI_bad.npy', NDSI_B_vals)

NDSI_H_vals = (NDSI_H, NDSI_H1, NDSI_H2, NDSI_H3, NDSI_H4)
np.save('NDSI_human.npy', NDSI_H_vals)

NDSI_M_vals = (NDSI_M, NDSI_M1, NDSI_M2, NDSI_M3, NDSI_M4)
np.save('NDSI_maybe.npy', NDSI_M_vals)

In [ ]:
#Loading

#ACI
ACI_good = np.load('ACI_good.npy', mmap_mode='r')
print(ACI_good.shape)


In [ ]:
ACI_G_vals = (ACI_G, ACI_G1, ACI_G2, ACI_G3, ACI_G4)
np.save('ACI_good.npz', ACI_G_vals)

In [ ]:
'''
#originally
ACI_G_vals = (np.arange(num_specs_g), (ACI_G, ACI_G1, ACI_G2, ACI_G3, ACI_G4))
ACI_G_file = np.array(ACI_G_vals)

ACI_B_vals = (ACI_B, ACI_B1, ACI_B2, ACI_B3, ACI_B4)
ACI_B_file = np.array([np.arange(num_specs_b), ACI_B_vals])

ACI_H_vals = (ACI_H, ACI_H1, ACI_H2, ACI_H3, ACI_H4)
ACI_H_file = np.array([np.arange(num_specs_h), ACI_H_vals])

ACI_M_vals = (ACI_M, ACI_M1, ACI_M2, ACI_M3, ACI_M4)
ACI_M_file = np.array([np.arange(num_specs_m), ACI_M_vals])

np.savez('ACI_vals.npz', ACI_G_file = ACI_G_file, ACI_B_file = ACI_B_file, ACI_H_file = ACI_H_file, ACI_M_file = ACI_M_file)
'''